# AI Model


## Settings

Our main settings are found in the config.yaml file, but you can also modify values here for convenience. Make sure you've run `pip install -U -r requirements.txt` in your environment before using this notebook!


In [1]:
import numpy as np
import random
import warnings
import torch
from utils.logging import warning_format
from utils.config import *

torch.set_float32_matmul_precision("medium")

warnings.formatwarning = warning_format

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
print(f"Seed: {SEED}")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Seed: 42


## Load Testing Data

This data is manually labeled, real-world laser scanned data. We will fetch this remotely if needed.


In [2]:
import os
from electrical_poles.data import download_data
from models.dataset import PointCloudDataset
from torch.utils.data import DataLoader

needs_validation = False
if not os.path.exists(TEST_DATA_PATH) or len(os.listdir(TEST_DATA_PATH)) == 0:
    print(
        "Testing data directory is empty or does not exist. New testing data will be downloaded."
    )
    download_data(out_dir=TEST_DATA_PATH, n_points=N_POINTS)
    needs_validation = True
else:
    print("Testing data directory found. Using existing testing data.")

test_dataset = PointCloudDataset(
    file_paths=[os.path.join(TEST_DATA_PATH, f) for f in os.listdir(TEST_DATA_PATH)],
    n_points=N_POINTS,
    n_classes=N_CLASSES,
)

if needs_validation:
    test_dataset.validate()

test_dataloader = DataLoader(
    test_dataset,
    batch_size=TEST_DATA_BATCH_SIZE,
    shuffle=False,
    num_workers=TEST_DATA_WORKERS,
    persistent_workers=True,
)

print(f"Testing dataset size: {len(test_dataset)}")

Testing data directory found. Using existing testing data.
Testing dataset size: 91


## Load our model

If you don't have a valid model ready, use train.py to train one!


In [3]:
import torch
from ai.pointnet_seg.model import PointNetSeg
from utils.config import *

segmenter = PointNetSeg.load_from_checkpoint(TRAIN_BEST_MODEL_PATH)
print("Model loaded")

Model loaded


# Demo

A little section here to show its labeling in action!


In [20]:
import numpy as np
import open3d as o3d
from utils.plot import point_cloud_figure
from utils.string import format_accuracy
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import display, HTML

segmenter.eval()
with torch.no_grad():
    inputs, labels = test_dataset[np.random.randint(0, len(test_dataset))]
    inputs = inputs.float().to(segmenter.device)
    labels = labels.to(segmenter.device)
    outputs, __, __ = segmenter(inputs.unsqueeze(0))
    _, predicted = torch.max(outputs.data, 1)

# Predicted point cloud
pr_pc = o3d.t.geometry.PointCloud()
pr_pc.point.positions = o3d.core.Tensor(
    inputs.squeeze(0).cpu().numpy(), dtype=o3d.core.Dtype.Float32
)
pr_pc.point.labels = predicted.squeeze(0).cpu().numpy()

# Ground truth point cloud
gt_pc = o3d.t.geometry.PointCloud()
gt_pc.point.positions = o3d.core.Tensor(
    inputs.squeeze(0).cpu().numpy(), dtype=o3d.core.Dtype.Float32
)
gt_pc.point.labels = labels.squeeze(0).cpu().numpy()

accuracy = (predicted == labels).sum().item() / N_POINTS
print(f"Accuracy: {format_accuracy(accuracy)}")
display(HTML("<div style='text-align: center;'><h1>Prediction</h1></div>"))
point_cloud_figure(
    pr_pc,
    xaxis=[-1.0, 1.0],
    yaxis=[-1.0, 1.0],
    zaxis=[-1.0, 1.0],
    title="Predicted",
    cmax=N_CLASSES - 1,
).show()
display(HTML("<div style='text-align: center;'><h1>Ground Truth</h1></div>"))
point_cloud_figure(
    gt_pc,
    xaxis=[-1.0, 1.0],
    yaxis=[-1.0, 1.0],
    zaxis=[-1.0, 1.0],
    title="Ground Truth",
    cmax=N_CLASSES - 1,
).show()

Accuracy: 99.02%
